<a href="https://colab.research.google.com/github/Veldora00/Veldora00/blob/main/hourly_forex_oanda_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install oandapyV20
!pip install fredapi
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
import oandapyV20
from oandapyV20 import API
from oandapyV20.endpoints import instruments
from datetime import timedelta
from fredapi import Fred

# OANDA API credentials
oanda_api_key = 'xx-xx'
oanda_account_id = 'zz'
oanda_api_url = 'https://api-fxtrade.oanda.com'

# FRED API key
fred_api_key = '1914a7e8b0b38709c172004439e86d0a'
fred = Fred(api_key=fred_api_key)

# Macroeconomic data series to fetch
macro_series = [
    'CPIAUCSL', 'UNRATE', 'GDP', 'PPIACO', 'FEDFUNDS', 'PAYEMS',
    'PCEPI', 'RSAFS', 'INDPRO', 'HOUST', 'BOPGSTB'
]

# Fetch macroeconomic data release dates
macro_dates = []
for series in macro_series:
    data_series = fred.get_series(series, observation_start='2023-11-01', observation_end='2024-08-01')
    macro_dates.extend(data_series.index.to_list())

# Convert macro dates to datetime format and remove duplicates
macro_dates = pd.to_datetime(macro_dates)
macro_dates = macro_dates.drop_duplicates()

# Function to fetch historical data from OANDA
def fetch_ohlc_data(symbol, granularity, count=5000):
    headers = {
        "Authorization": f"Bearer {oanda_api_key}",
        "Content-Type": "application/json"
    }
    api = API(access_token=oanda_api_key, environment="live", headers=headers)
    params = {
        "granularity": granularity,
        "count": count
    }
    r = instruments.InstrumentsCandles(instrument=symbol, params=params)
    try:
        response = api.request(r)
    except oandapyV20.exceptions.V20Error as e:
        print(f"Error fetching data: {e}")
        return None

    if 'candles' not in response:
        print("Error fetching data: Candles not found in response.")
        return None

    candles = response['candles']
    df = pd.DataFrame([{
        'time': candle['time'],
        'open': float(candle['mid']['o']),
        'high': float(candle['mid']['h']),
        'low': float(candle['mid']['l']),
        'close': float(candle['mid']['c'])
    } for candle in candles])
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    return df


def backtest(data, symbol, macro_dates=None, initial_balance=10000, pip_value=0.0001, risk_per_trade=0.005, sl_pips=10, move_to_breakeven_pips=20):
    balance = initial_balance
    equity_curve = []
    drawdown = []
    max_balance = initial_balance
    in_position = False
    entry_price = 0
    sl = 0
    tp = 0
    position_size = 0
    positions = []
    total_trades = 0
    profit_trades = 0
    loss_trades = 0
    trades = []
    debug_log = []

    for i in range(len(data)):
        current_date = data.index[i]
        current_price = data['close'].iloc[i]

        # Skip macroeconomic dates
        if macro_dates is not None and current_date in macro_dates:
            debug_log.append(f"Skipping due to macro date: {current_date}")
            continue

        # Calculate the dollar risk for the trade
        dollar_risk = balance * risk_per_trade

        # Check if in position and close position if TP or SL is hit
        if in_position:
            if positions[-1][1] == 'Short':
                if current_price >= sl:  # SL hit
                    trade_result = position_size * (entry_price - sl)
                    balance += trade_result
                    trades.append([symbol, current_date, 'Short', entry_price, sl, sl, tp, trade_result])
                    debug_log.append(f"Short SL hit at {current_date} for {symbol}. Price: {current_price}, SL: {sl}")
                    in_position = False
                    loss_trades += 1
                elif current_price <= tp:  # TP hit
                    trade_result = position_size * (entry_price - tp)
                    balance += trade_result
                    trades.append([symbol, current_date, 'Short', entry_price, tp, sl, tp, trade_result])
                    debug_log.append(f"Short TP hit at {current_date} for {symbol}. Price: {current_price}, TP: {tp}")
                    in_position = False
                    profit_trades += 1
                elif entry_price - current_price >= move_to_breakeven_pips * pip_value:
                    sl = entry_price  # Move SL to break-even
                    debug_log.append("Short position SL moved to break-even")
            elif positions[-1][1] == 'Long':
                if current_price <= sl:  # SL hit
                    trade_result = position_size * (sl - entry_price)
                    balance += trade_result
                    trades.append([symbol, current_date, 'Long', entry_price, sl, sl, tp, trade_result])
                    debug_log.append(f"Long SL hit at {current_date} for {symbol}. Price: {current_price}, SL: {sl}")
                    in_position = False
                    loss_trades += 1
                elif current_price >= tp:  # TP hit
                    trade_result = position_size * (tp - entry_price)
                    balance += trade_result
                    trades.append([symbol, current_date, 'Long', entry_price, tp, sl, tp, trade_result])
                    debug_log.append(f"Long TP hit at {current_date} for {symbol}. Price: {current_price}, TP: {tp}")
                    in_position = False
                    profit_trades += 1
                elif current_price - entry_price >= move_to_breakeven_pips * pip_value:
                    sl = entry_price  # Move SL to break-even
                    debug_log.append("Long position SL moved to break-even")

        # Enter short position at swing high
        if not in_position and data['Signal'].iloc[i] == -1:
            in_position = True
            entry_price = current_price
            sl = data['Swing_High'].iloc[i] + sl_pips * pip_value
            tp = entry_price - 3 * (sl - entry_price)
            position_size = dollar_risk / abs(entry_price - sl)
            positions.append((current_date, 'Short', entry_price, sl, tp, position_size))
            total_trades += 1
            debug_log.append(f"Entered Short: Entry={entry_price}, SL={sl}, TP={tp}, Size={position_size}")

        # Enter long position at swing low
        elif not in_position and data['Signal'].iloc[i] == 1:
            in_position = True
            entry_price = current_price
            sl = data['Swing_Low'].iloc[i] - sl_pips * pip_value
            tp = entry_price + 3 * (entry_price - sl)
            position_size = dollar_risk / abs(entry_price - sl)
            positions.append((current_date, 'Long', entry_price, sl, tp, position_size))
            total_trades += 1
            debug_log.append(f"Entered Long: Entry={entry_price}, SL={sl}, TP={tp}, Size={position_size}")

        # Update equity and drawdown
        equity_curve.append(balance)
        if balance > max_balance:
            max_balance = balance
        drawdown.append((max_balance - balance) / max_balance)

    trades_df = pd.DataFrame(trades, columns=['Symbol', 'Date', 'Type', 'Entry', 'Exit', 'SL', 'TP', 'P/L'])
    return equity_curve, drawdown, balance, trades_df, debug_log
    # Inside the backtest function


# Signal Generation with Debug Logging
def generate_signals(data):
    order = 20
    data['Swing_Low'] = data.iloc[argrelextrema(data['close'].values, np.less_equal, order=order)[0]]['close']
    data['Swing_High'] = data.iloc[argrelextrema(data['close'].values, np.greater_equal, order=order)[0]]['close']

    # Initialize signal column
    data['Signal'] = 0

    # Generate signals based on swing highs/lows
    for i in range(1, len(data)):
        if not np.isnan(data['Swing_High'].iloc[i]):
            data.at[data.index[i], 'Signal'] = -1  # Go Short at Swing High
        elif not np.isnan(data['Swing_Low'].iloc[i]):
            data.at[data.index[i], 'Signal'] = 1  # Go Long at Swing Low

    # Debug logs to verify signals
    signal_counts = data['Signal'].value_counts()
    print(f"Signals generated: Long = {signal_counts.get(1, 0)}, Short = {signal_counts.get(-1, 0)}")

    return data

# Apply signal generation and backtest
all_trades = []
symbols = ['EUR_GBP', 'EUR_AUD', 'EUR_NZD', 'EUR_CHF', 'EUR_CAD',
               'GBP_AUD', 'GBP_NZD', 'GBP_CHF', 'GBP_CAD',
               'AUD_NZD', 'AUD_CHF', 'AUD_CAD',
               'NZD_CHF', 'NZD_CAD',
               'CHF_CAD']
final_balances = []
for symbol in symbols:
    # Fetch historical data
    data = fetch_ohlc_data(symbol, granularity='H4', count=5000)

    if data is None:
        print(f"Failed to fetch data for {symbol}")
        continue

    # Generate signals
    data = generate_signals(data)

    # Run backtest
    initial_balance = 218.08
    equity_curve, drawdown, final_balance, trades_df, debug_log = backtest(data, symbol, macro_dates, initial_balance)
    final_balances.append(final_balance)
    all_trades.append(trades_df)

    # Print results for each symbol
    print(f'\nResults for {symbol}:')
    print(f'Net Profit/Loss: {final_balance - initial_balance:.2f}')
    max_drawdown = max(drawdown) if drawdown else 0
    print(f'Max Drawdown: {max_drawdown:.2%}')
    print(f'Total Equity: {equity_curve[-1]:.2f}')
    print(f'Total Trades: {len(trades_df)}')
    print(f'Profitable Trades: {len(trades_df[trades_df["P/L"] > 0])}')
    print(f'Losing Trades: {len(trades_df[trades_df["P/L"] <= 0])}')

    # Plotting
    plt.figure(figsize=(14, 7))

    # Plot close price and signals
    plt.subplot(2, 1, 1)
    plt.plot(data['close'], label='Close Price', alpha=0.5)
    plt.scatter(data.index, data['Swing_Low'], color='red', label='Swing Lows')
    plt.scatter(data.index, data['Swing_High'], color='green', label='Swing Highs')
    plt.plot(data[data['Signal'] == 1].index, data['close'][data['Signal'] == 1], '^', markersize=10, color='g', lw=0, label='Long Signal')
    plt.plot(data[data['Signal'] == -1].index, data['close'][data['Signal'] == -1], 'v', markersize=10, color='r', lw=0, label='Short Signal')
    plt.title(f'{symbol} Swing High and Low Strategy')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()

    # Plot equity curve and drawdown
    plt.subplot(2, 1, 2)
    plt.plot(data.index, equity_curve, label='Equity Curve')
    plt.fill_between(data.index, np.array(equity_curve) - np.array(drawdown) * np.array(equity_curve), equity_curve, color='red', alpha=0.3, label='Drawdown')
    plt.title(f'Equity Curve and Drawdown for {symbol}')
    plt.xlabel('Date')
    plt.ylabel('Equity')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Concatenate all trades into a single DataFrame and save to CSV
all_trades_df = pd.concat(all_trades)
all_trades_df.to_csv('all_trades_oanda.csv', index=False)
print("All trades saved to all_trades_oanda.csv")

# Display combined performance metrics
final_metrics = {
    'Initial Balance': initial_balance,
    'Final Balance': sum(final_balances),
    'Net Profit': sum(final_balances) - initial_balance * len(symbols)
}

print(final_metrics)

ERROR:oandapyV20.oandapyV20:request https://api-fxtrade.oanda.com/v3/instruments/EUR_GBP/candles failed [401,{"errorMessage":"Insufficient authorization to perform request."}]
ERROR:oandapyV20.oandapyV20:request https://api-fxtrade.oanda.com/v3/instruments/EUR_AUD/candles failed [401,{"errorMessage":"Insufficient authorization to perform request."}]


Error fetching data: {"errorMessage":"Insufficient authorization to perform request."}
Failed to fetch data for EUR_GBP
Error fetching data: {"errorMessage":"Insufficient authorization to perform request."}
Failed to fetch data for EUR_AUD


ERROR:oandapyV20.oandapyV20:request https://api-fxtrade.oanda.com/v3/instruments/EUR_NZD/candles failed [401,{"errorMessage":"Insufficient authorization to perform request."}]
ERROR:oandapyV20.oandapyV20:request https://api-fxtrade.oanda.com/v3/instruments/EUR_CHF/candles failed [401,{"errorMessage":"Insufficient authorization to perform request."}]


Error fetching data: {"errorMessage":"Insufficient authorization to perform request."}
Failed to fetch data for EUR_NZD
Error fetching data: {"errorMessage":"Insufficient authorization to perform request."}
Failed to fetch data for EUR_CHF


ERROR:oandapyV20.oandapyV20:request https://api-fxtrade.oanda.com/v3/instruments/EUR_CAD/candles failed [401,{"errorMessage":"Insufficient authorization to perform request."}]
ERROR:oandapyV20.oandapyV20:request https://api-fxtrade.oanda.com/v3/instruments/GBP_AUD/candles failed [401,{"errorMessage":"Insufficient authorization to perform request."}]


...

ERROR:oandapyV20.oandapyV20:request https://api-fxtrade.oanda.com/v3/instruments/NZD_CHF/candles failed [401,{"errorMessage":"Insufficient authorization to perform request."}]
ERROR:oandapyV20.oandapyV20:request https://api-fxtrade.oanda.com/v3/instruments/NZD_CAD/candles failed [401,{"errorMessage":"Insufficient authorization to perform request."}]


Error fetching data: {"errorMessage":"Insufficient authorization to perform request."}
Failed to fetch data for NZD_CHF
Error fetching data: {"errorMessage":"Insufficient authorization to perform request."}
Failed to fetch data for NZD_CAD


ERROR:oandapyV20.oandapyV20:request https://api-fxtrade.oanda.com/v3/instruments/CHF_CAD/candles failed [401,{"errorMessage":"Insufficient authorization to perform request."}]


Error fetching data: {"errorMessage":"Insufficient authorization to perform request."}
Failed to fetch data for CHF_CAD


ValueError: No objects to concatenate